In [ ]:
import os
import glob

In [ ]:
import pickle

In [ ]:
import weaviate

# Data

In [ ]:
import tensorflow as tf

In [ ]:
def read(x):
    return pd.read_json(x, lines=True)

In [ ]:
joined_files = os.path.join("D:\\jupyter\\big data\\project\\model\\processed_data\\combined_data", "*.json")
joined_list = glob.glob(joined_files)
sof_df = pd.concat(map(read, joined_list), ignore_index=True)


In [ ]:
sof_df = pd.read_json("D:\\jupyter\\big data\\project\\model\\tag-duplicate-questions\\part-00000-9a6a4bc5-98e9-4b82-80e4-d7396c7d600e-c000.json",lines=True)

In [ ]:
sof_df

In [ ]:
questions = (list(sof_df.iloc[:,0])+list(sof_df.iloc[:,0]))

In [ ]:
pickle.dump(questions, open('questions_list.p', 'wb'))

In [5]:
questions =  pickle.load(open("questions_list.p", "rb"))

d:\jupyter\big data\project\spark-test\venv_tf_1\lib\site-packages\ipykernel_launcher.py:1: ResourceWarning: unclosed file <_io.BufferedReader name='questions_list.p'>
  """Entry point for launching an IPython kernel.


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(questions)

# Embedding model

In [9]:
import tensorflow_hub as hub
embed = hub.load("D:/jupyter/big data/project/spark-test/use")

In [ ]:
sbert_model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [ ]:
def gen_sbert_embed(text):
    vectors = model.encode(text)
    return vectors

In [7]:
def gen_use_embedding(questions):
    embeddings = embed(questions)
    return embeddings

In [ ]:
emd_dataset = dataset.map(lambda x: (x,gen_use_embedding([x])))

In [ ]:
len(emd_dataset)

In [20]:
questions

['Construct callable function from string',
 'How to override bad symlink with python',
 'forking a process in window',
 'How to implement structure data type in python',
 'How to sort the list in Python?',
 "Why is class variable' s type the supclass's type?",
 'Why am I using a @classmethod instead of a normal instance method',
 'How to add keys and values from two dictionaries to third without mixing between them?',
 'Replacing a line in an already opened file python',
 'how to run python in iphone?',
 'How to write long arithmetic expressions in several lines in python?',
 'round() does not trucate float',
 'Python Use Switch Case?',
 'Importing Python file from PostgreSQL stored procedure',
 'Is there any python based web framework that follows MVC?',
 'Variables accessed by a function within a class',
 'Getting class name without instanting an object',
 'python unittest - fails even without any test',
 'How can I split a string with a number at the end?',
 'Python: What is for ..

# Weaviate

In [ ]:
import numpy as np

In [ ]:
def init_weaviate_schema():
    schema = {
        "classes": [{
                "class": "Questions",
                "vectorizer": "none", 
                "properties": [{
                    "name": "question_text",
                    "dataType": ["text"],
                }]
        }]
    }

    client.schema.delete_all()
    client.schema.create(schema)

In [17]:
def search(query, limit=3):
    vec = gen_use_embedding([query])
    near_vec = {"vector": vec}
    res = client.query.get("Questions", ["question_text", "_additional {certainty}"]).with_near_vector(near_vec).with_limit(limit).do()
    for q in res["data"]["Get"]["Questions"]:
        print("{:.4f}: {}".format(q["_additional"]["certainty"], q["question_text"]))
        print('---')

In [ ]:
client = weaviate.Client("http://localhost:8090")

In [ ]:
init_weaviate_schema()

In [ ]:
client.batch.configure(
  batch_size=None, 
)

In [ ]:
len(emd_dataset)

In [ ]:
batch = client.batch()
count=0
for q,v in emd_dataset:

    props = {
        "question_text": q.numpy().decode("utf-8")
    }
    batch.add_data_object(props, "Questions",vector = v.numpy())
    if count==10:
        batch.create_objects()
        batch = client.batch()
        count = 0

batch.create_objects()


In [ ]:
search("file handling in python", 3)

In [ ]:
how to take input from user
can we get user profile data from Facebook API
databricks error when using apache spark
memory issues with gradient decent
how to add facebook authentication
can we convert rdd to dataframes
regression vs classification
weighted cross entropy